# Limpieza y Transformación de Datos: Uniformidad y Consistencia

Este notebook asegura que todas las películas y series tengan comentarios válidos o instrucciones claras si no los hay, eliminando valores faltantes y homogenizando la información.


In [2]:
import pandas as pd
import numpy as np

C:\Users\Arnaud\AppData\Local\Temp\ipykernel_37832\2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
#Tablon de las reseñas de las películas
df = pd.read_csv("C:\\Users\\Arnaud\\Desktop\\Netflix\\Raw_data\\df_reseñas.csv",index_col=0)
df_clustered = pd.read_csv("C:\\Users\\Arnaud\\Desktop\\Netflix\\Raw_data\\df_clustered.csv",index_col=0)

#### Info del Tablón con todas las reseñas

In [138]:
df.head()

,title,Título_commentario,Comentario
0,Five Came Back: The Reference Films,Compelling examination of propaganda,An American documentary; A film about five suc...
1,Five Came Back: The Reference Films,Riveting and a must see,"Just finished watching""Five came back"", a trul..."
2,Five Came Back: The Reference Films,Spectacular,I have never seen any film made by those five ...
3,Five Came Back: The Reference Films,Breathtaking,I am an AVID WW2 history buff. I've watched co...
4,Five Came Back: The Reference Films,Superb documentary series,During World War 2 several movie directors joi...


In [139]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 113179 entries, 0 to 113178
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   title               113172 non-null  object
 1   Título_commentario  113178 non-null  object
 2   Comentario          113179 non-null  object
dtypes: object(3)
memory usage: 3.5+ MB


In [140]:
#Cambiar el nombre de las columnas
df.rename(columns={"Título_commentario":"title_comment","Comentario":"comment"},inplace=True)  

### Borramos las filas con comentario no disponible y que el título disponga de mas comentarios.
Comment = *`"Comentario no disponible"`*  son comentarios que incluyen **spoilers** por lo que no se han scrapeado.

In [141]:
df[df["comment"] == "Comentario no disponible"].head()

,title,title_comment,comment
99141,Alexandria… Why?,A tumble of unreliable memories,Comentario no disponible
99144,Dark Waters,"""We should want to nail Dupont!""",Comentario no disponible
99147,Dark Waters,Thank you for the fight,Comentario no disponible
99159,Dark Waters,Disturbing Story,Comentario no disponible
99161,Dark Waters,Eye Opening Tale,Comentario no disponible


In [142]:
# Identificar los títulos que tienen al menos un comentario válido
titles_with_valid_comments = df[df["comment"] != "Comentario no disponible"]["title"].unique()

# Limpiar los comentarios
def process_row(row, valid_titles):
    if row["title"] in valid_titles and row["comment"] == "Comentario no disponible":
        return row["title_comment"]  # Copiar el title_comment al comentario
    elif row["title"] not in valid_titles and row["comment"] == "Comentario no disponible":
        return "Este título no tiene suficientes comentarios para ser analizado. No hay pros y contras que analizar, solo haz el breve resumen del título."
    return row["comment"]  # Mantener el comentario actual

# Actualizar los comentarios
df["comment"] = df.apply(lambda row: process_row(row, titles_with_valid_comments), axis=1)

In [143]:
# Comentarios que son iguales a sus títulos
df[df["comment"] ==(df["title_comment"])].info()

<class 'pandas.core.frame.DataFrame'>
Index: 19043 entries, 20 to 112850
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          19043 non-null  object
 1   title_comment  19043 non-null  object
 2   comment        19043 non-null  object
dtypes: object(3)
memory usage: 595.1+ KB


### Remplazamos los comentarios de los títulos que no disponen de ningúna reseña por una instrucción clara para que la LLM sepa como tratar el título
title_comment = *`"No disponible"`* son títulos poco conocídos que no tienen ningún comentario para scrapear

In [144]:
# títulos que no tienen comentarios
df[df["title_comment"] == "No disponible"].head()

,title,title_comment,comment
111516,Thiago Ventura: POKAS,No disponible,Esta película no tiene reseñas disponibles.
111567,Los Tigres del Norte at Folsom Prison,No disponible,Esta película no tiene reseñas disponibles.
111568,What the F* Is Going On?,No disponible,Esta película no tiene reseñas disponibles.
111569,J-Style Trip,No disponible,Esta película no tiene reseñas disponibles.
111591,Making Unorthodox,No disponible,Esta película no tiene reseñas disponibles.


In [145]:
# Actualizar los comentarios de los títulos que no tienen comentarios por una instrucción
df.loc[df["title_comment"] == "No disponible", "comment"] = (
    "Este titulo no tiene suficientes comentarios para ser analizado. No Hay pros y contras que analizar, solo haz el breve resumen del título.")

In [146]:
# Títulos que no tienen comentarios, pero si tienen instrucción
df[df["comment"] == "Este titulo no tiene suficientes comentarios para ser analizado. No Hay pros y contras que analizar, solo haz el breve resumen del título."].info()

<class 'pandas.core.frame.DataFrame'>
Index: 545 entries, 111516 to 113178
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          545 non-null    object
 1   title_comment  545 non-null    object
 2   comment        545 non-null    object
dtypes: object(3)
memory usage: 17.0+ KB


### Valores faltantes

1. **Títulos faltantes:** Se detectaron filas con valores nulos en la columna de títulos. Estas filas fueron eliminadas porque los títulos eran repetidos y no aportaban información adicional.

2. **Título de comentario vacío:** Se encontró un título de comentario vacío en la columna `title_comment`. Este fue reemplazado por el texto `"Título del comentario no disponible"` para garantizar la uniformidad de los datos.

In [147]:
df.isnull().sum()

title            7
title_comment    1
comment          0
dtype: int64

In [148]:
# Eliminar las filas que tengan valores nulos en la columna "title" ya que son repetidos
df.dropna(subset=["title"], inplace=True)

# Remplazamos el título del comentario  por "Título del comentario no disponible"
df[df["title_comment"].isnull()] = "Título del comentario no disponible"

In [149]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 113172 entries, 0 to 113178
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   title          113172 non-null  object
 1   title_comment  113172 non-null  object
 2   comment        113172 non-null  object
dtypes: object(3)
memory usage: 3.5+ MB


#### Exportar csv el tablón con los datos limpios y transformados.

In [150]:
df.to_csv("C:\\Users\\Arnaud\\Desktop\\Netflix\\Raw_data\\reseñas_tablon_final.csv")